### Preparacion comun

In [16]:
import numpy as np

A = np.random.rand(1000, 3)  # Usamos tamaños más pequeños para hacerlo manejable
B = np.random.rand(1000, 3)

### Bucle for tradicional

In [17]:
import time
from scipy.spatial import distance

start_time = time.time()

distancias_for = []
for a in A:
    for b in B:
        dist = np.sqrt(np.sum((a - b) ** 2))
        distancias_for.append(dist)

end_time = time.time()
print(f"Bucle `for` original tardó: {end_time - start_time} segundos")

Bucle `for` original tardó: 3.8682610988616943 segundos


### Versión vectorizada con NumPy

In [18]:
import numpy as np
start_time = time.time()

diff = A[:, np.newaxis, :] - B[np.newaxis, :, :]
distancias_vectorizadas = np.sqrt(np.sum(diff**2, axis=2))

end_time = time.time()
print(f"Vectorización con NumPy tardó: {end_time - start_time} segundos")

Vectorización con NumPy tardó: 0.07084441184997559 segundos


### Paralelización con joblib

In [19]:
from joblib import Parallel, delayed

def calcular_distancias_joblib(a, B):
    return [np.sqrt(np.sum((a - b) ** 2)) for b in B]

start_time = time.time()

resultados_joblib = Parallel(n_jobs=-1)(delayed(calcular_distancias_joblib)(a, B) for a in A)

end_time = time.time()
print(f"Paralelización con `joblib` tardó: {end_time - start_time} segundos")


Paralelización con `joblib` tardó: 0.9430487155914307 segundos


### Paralelización con multiprocessing

In [20]:
from multiprocessing import Pool

def calcular_distancias_multiprocessing(a, B):
    return [np.sqrt(np.sum((a - b) ** 2)) for b in B]

if __name__ == '__main__':
    start_time = time.time()

    with Pool(processes=4) as pool:
        resultados_multiprocessing = pool.starmap(calcular_distancias_multiprocessing, [(a, B) for a in A])

    end_time = time.time()
    print(f"Paralelización con `multiprocessing` tardó: {end_time - start_time} segundos")

Paralelización con `multiprocessing` tardó: 1.6318132877349854 segundos


### Simulaciones

In [24]:
import numpy as np
import time
from joblib import Parallel, delayed
import multiprocessing

# Definiciones de las funciones de ejemplo

def for_loop(A, B):
    result = np.zeros_like(A)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            result[i, j] = A[i, j] + B[i, j]  # Operación simple de ejemplo
    return result

def vectorization(A, B):
    return A + B

def add(a,b):
    return a+b

def joblib_parallel(A, B):
    # Esta es una función de ejemplo para la operación paralela con Joblib
    
    result = Parallel(n_jobs=-1)(delayed(add)(A[i], B[i]) for i in range(len(A)))
    return np.array(result)

def multiprocessing_parallel(A, B):
    # Definición similar a joblib pero usando multiprocessing
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    result = pool.starmap(add, [(A[i], B[i]) for i in range(len(A))])
    pool.close()
    pool.join()
    return np.array(result)

# Asegúrate de definir A y B con el tamaño deseado
A = np.random.rand(10000, 3)
B = np.random.rand(10000, 3)

# Número de simulaciones y listas para tiempos
num_simulaciones = 5
tiempos_for_loop = []
tiempos_vectorizacion = []
tiempos_joblib = []
tiempos_multiprocessing = []

for _ in range(num_simulaciones):
    # Repetimos el bloque de código de medición de tiempos para cada método
    start_time = time.time()
    for_loop(A, B)
    tiempos_for_loop.append(time.time() - start_time)
    
    start_time = time.time()
    vectorization(A, B)
    tiempos_vectorizacion.append(time.time() - start_time)
    
    start_time = time.time()
    joblib_parallel(A, B)
    tiempos_joblib.append(time.time() - start_time)
    
    start_time = time.time()
    multiprocessing_parallel(A, B)
    tiempos_multiprocessing.append(time.time() - start_time)

In [28]:
import matplotlib.pyplot as plt
import pandas as pd

# Crear DataFrame para análisis
df_tiempos = pd.DataFrame({
    'For Loop': tiempos_for_loop,
    'Vectorización': tiempos_vectorizacion,
    'Joblib': tiempos_joblib,
    'Multiprocessing': tiempos_multiprocessing
})

# Estadísticos básicos
estadisticos_basicos = df_tiempos.describe()

# Gráfico de los tiempos de ejecución
plt.figure(figsize=(10, 6))
df_tiempos.boxplot()
plt.title('Comparación de Tiempos de Ejecución')
plt.ylabel('Tiempo (s)')
plt.grid(True)
plt.show(), estadisticos_basicos


ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.